In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

In [2]:
feature_dataset1 = pd.read_csv('feature_dataset.csv')
feature_dataset1.drop(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r',
                       's','t','u','v','w','x','y','z','entropy','length','tld'], axis=1, inplace=True)
feature_dataset1 = feature_dataset1.drop_duplicates()

In [3]:
feature_dataset2 = pd.read_csv('feature_dataset2.csv')
feature_dataset2.drop(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r',
                       's','t','u','v','w','x','y','z','label','tld'], axis=1, inplace=True)
#feature_dataset2.drop(['label','tld'], axis=1, inplace=True)

In [4]:
feature_frame = pd.merge(feature_dataset1, feature_dataset2,  how='inner', on=['domains','family'])

In [5]:
feature_frame = feature_frame.fillna(0)

In [6]:
feature_frame.count()

domains               1256032
family                1256032
label                 1256032
ratio                 1256032
diff_char             1256032
pct_coverage          1256032
digits                1256032
-                     1256032
bi-gram_normality     1256032
tri-gram_normality    1256032
length                1256032
entropy               1256032
distinct              1256032
vowel                 1256032
digit                 1256032
dash                  1256032
gram1                 1256032
gram2                 1256032
gram3                 1256032
gram4                 1256032
gram5                 1256032
percentage2           1256032
percentage3           1256032
percentage4           1256032
percentage5           1256032
hit                   1256032
percentage            1256032
maxlength             1256032
maxhit                1256032
maxcoverage           1256032
cmaxlength            1256032
cmaxhit               1256032
cmaxcoverage          1256032
full      

In [ ]:
msk = np.random.rand(len(feature_frame)) < 0.70
train = feature_frame[msk]
test = feature_frame[~msk]
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
features = feature_frame.columns[4:]

In [ ]:
#clf = RandomForestClassifier(n_jobs=50)
clf = GradientBoostingClassifier(n_estimators=200,
                             max_depth=5,
                             init=None,
                             verbose=2)
y, _ = pd.factorize(train['label'])

In [ ]:
clf.fit(train[features], y)
test['prediction'] = clf.predict(test[features])
predicted_proba = clf.predict_proba(test[features])
print(features)

In [ ]:
bad_probability = [y for x, y in predicted_proba]
test['bad_prob'] = bad_probability

In [ ]:
y_test, _ = pd.factorize(test['label'])
print(metrics.confusion_matrix(y_test, test['prediction']))

In [ ]:
print(metrics.classification_report(y_test, test['prediction']))

In [ ]:
print(metrics.accuracy_score(y_test, test['prediction']))

In [ ]:
clf.feature_importances_

In [ ]:
features

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
objects = clf.feature_importances_
y_pos = np.arange(len(objects))
b1 = plt.bar(y_pos, objects, align='center', alpha=0.5)
plt.ylabel('Score')
plt.title('Feature Importance')
plt.show()

In [ ]:
bluecoat1 = pd.read_csv('bluecoat.csv')
bluecoat1.drop(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r',
                       's','t','u','v','w','x','y','z','entropy','length','tld'], axis=1, inplace=True)
bluecoat1 = bluecoat1.drop_duplicates()

In [ ]:
bluecoat2 = pd.read_csv('bluecoat2.csv')
bluecoat2.drop(['tld'], axis=1, inplace=True)

In [ ]:
bluecoat_frame = pd.merge(bluecoat1, bluecoat2,  how='inner', on=['domains'])

In [ ]:
bluecoat_frame = bluecoat_frame.fillna(0)

In [ ]:
bluecoat_frame['prediction'] = clf.predict(bluecoat_frame[features])
predicted_proba = clf.predict_proba(bluecoat_frame[features])
print(features)

In [ ]:
bad_probability = [y for x, y in predicted_proba]
bluecoat_frame['bad_prob'] = bad_probability

In [ ]:
filtered = bluecoat_frame[['domains','bad_prob','prediction']]
filtered1 = filtered[filtered['prediction']==1]
filtered2 = filtered1.sort(['bad_prob'], ascending=False).reset_index(drop=True)
sure_bad = filtered2[:4]

In [ ]:
filtered11 = filtered[filtered['prediction']==0]
filtered22 = filtered11.sort(['bad_prob'], ascending=True).reset_index(drop=True)
sure_good = filtered22[20000:20005]

In [ ]:
filtered111 = filtered[filtered['prediction']==1]
filtered222 = filtered111.sort(['bad_prob'], ascending=True).reset_index(drop=True)
on_the_border1 = filtered222[:5]

In [ ]:
filtered1111 = filtered[filtered['prediction']==1]
filtered2222 = filtered1111.sort(['bad_prob'], ascending=False).reset_index(drop=True)
#sure_bad = filtered2[:5]
def_good0 = filtered2222[filtered2222['domains']=='rsvpify']
def_good1 = filtered2222[filtered2222['domains']=='pwcgov']

In [ ]:
dict_dga = pd.DataFrame({'domains': ['drinkteach', 'threeusual'],
                         'bad_prob': [0.171011, 0.126560],
                         'prediction': [0, 0]
                        })

In [ ]:
demo_data = pd.concat([sure_bad, sure_good, on_the_border1, def_good0, def_good1, dict_dga], ignore_index=True)

#We are good at predicting these bad guys

In [ ]:
sure_bad

#We are good at predicting these good guys

In [ ]:
sure_good

#We are having slight trouble with these guys but it should be an easy fix

In [ ]:
on_the_border1

#We are working separately for these guys

In [ ]:
pd.concat([def_good0, def_good1, dict_dga], ignore_index=True)

In [ ]:
mu = np.mean(test['bad_prob'])
sigma = np.std(test['bad_prob'])
print(mu)
print(sigma)

In [ ]:
import matplotlib.mlab as mlab

In [ ]:
x = mu + sigma * np.random.randn(10000)

num_bins = 376
arr = test['bad_prob']
arr = [x*100 for x in arr]
# the histogram of the data
plt.hist(arr, num_bins, normed=0, facecolor='green', alpha=0.5)
# add a 'best fit' line
#y = mlab.normpdf(bins, mu, sigma)
plt.plot(bins, y, 'r--')
plt.xlabel('Probability')
plt.ylabel('Domains')
plt.title(r'Histogram')

# Tweak spacing to prevent clipping of ylabel
plt.subplots_adjust(left=0.15)
plt.show()

In [ ]:
import scipy.special

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

In [ ]:
p1 = figure(title="Probability Distribution",tools="save",
            background_fill="#E8DDCB")

arr = [prob for prob in test['bad_prob'] if prob >= 0.95]

hist, edges = np.histogram(arr, density=True, bins=376930)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")

p1.legend.location = "top_left"
p1.xaxis.axis_label = 'Probability'
p1.yaxis.axis_label = 'Domains'

show(p1)

In [ ]:
test[test['bad_prob']>=0.95]